# GSM-EPD

In [ ]:
%matplotlib widget
from bmcs_matmod.api import GSM1D_EPD
import matplotlib.pylab as plt
import sympy as sp
import numpy as np
from IPython.display import display, Math, Markdown
gsm = GSM1D_EPD()
display(Markdown(gsm.latex_potentials()))

## Linearly ascending loading

In [ ]:
E_val = 30e+3  # Young's modulus in MPa
K_val = 1e+3  # Bulk modulus in MPa
f_c_val = 15 # Strength in MPa
eta_vp_val = 10e+4  # Viscosity in MPa s
eta_ve_val = 10e+4  # Viscosity in MPa s

# mparams = {gsm.E:10, gsm.c:1, gsm.r:1, gsm.S:10, gsm.eps_0:1}

mparams = dict(E=E_val, K=K_val, f_c=f_c_val, S=0.0001, r=1, c=1,
            eps_0=0.001, eta_ve=eta_ve_val, eta_vp=eta_vp_val)

f_time = 10  # Final time in secondsfinal_time = 1
n_t = 151
t_t = np.linspace(0, f_time, n_t)
eps_max = 0.0025
eps_n_t = np.linspace(0, eps_max, n_t)

response = gsm.get_response(eps_n_t, t_t, **mparams)
t_t_ec, eps_ta_ec, sig_ta_ec, Eps_t_ec, Sig_t_ec, iter_t_ec, lam_t_ec, (d_t_t_ec, d_eps_ta_ec) = response
eps_t_ec = eps_ta_ec[:, 0]
sig_t_ec = sig_ta_ec[:, 0, 0]
# use the stress history to reproduce the response using stress control
response = gsm.get_G_response(sig_t_ec, t_t, **mparams)
t_t_sc, eps_ta_sc, sig_ta_sc, Eps_t_sc, Sig_t_sc, iter_t_sc, lam_t_sc, (d_t_t_sc, d_eps_ta_sc) = response
iter_t = iter_t_sc[:, 0]
eps_t_sc = eps_ta_sc[:, 0, 0]
sig_t_sc = sig_ta_sc[:, 0]
eps_t_ec.shape, sig_t_ec.shape, eps_t_sc.shape, sig_t_sc.shape

In [ ]:
F_engine = gsm.F_engine
G_engine = gsm.G_engine

## Monotonic strain-driven loading

In [ ]:
mparams = dict(E = 20, K = 10, S = 1, c = 1, r = 1, eps_0 = 0, f_c = 15)
n_t = 151
n_I = 1
eps_max = 1.3
eps_max = 5
final_time = 10
t_t = np.linspace(0, final_time, n_t)
eps_n_t = np.linspace(0, eps_max, n_t)
response = gsm.get_response(eps_n_t, t_t, **mparams)
t_t, eps_ta, sig_ta, Eps_t, Sig_t, iter_t, lam_t, (d_t_t, d_eps_ta) = response
# Plot viscoplastic strain and damage along time
eps_t = eps_ta[:, 0]
sig_t = sig_ta[:, 0, 0]
eps_p_t, omega_t, z_t = Eps_t[:, 0, :].T
sig_p_t, Y_t, Z_t = Sig_t[:, 0, :].T
eps_t.shape, sig_t.shape
sig_t_strain = np.copy(sig_t)

In [ ]:
fig, ((ax_sig, ax_omega), (ax_lam, ax_z)) = plt.subplots(2, 2, figsize=(12, 8))

# First row: Stress-strain and damage plots
ax_iter = ax_sig.twinx()
ax_eps = ax_omega.twinx()

ax_sig.plot(eps_t, sig_t, color='cadetblue', label=r'$\varepsilon$')
ax_sig.plot(eps_t-eps_p_t, sig_t, ls='dashed', color='cadetblue', label=r'$\varepsilon-\varepsilon^\mathrm{ve}$')
ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')
ax_sig.legend()

ax_iter.plot(eps_t, iter_t, ls='dashed', color='gray', lw=0.5)
ax_iter.set_ylabel(r'$k$', color='gray')

ax_omega.plot(t_t, omega_t, label='Damage', color='brown')
ax_eps.plot(t_t, eps_t, label=r'$\varepsilon$', color='cadetblue', lw=1)
ax_eps.plot(t_t, eps_p_t, label=r'$\varepsilon_\mathrm{vp}$', color='cadetblue', ls='dashed', lw=1)
ax_eps.legend()
ax_omega.set_xlabel(r'$t$')
ax_omega.set_ylabel(r'$\omega$', color='brown')
ax_eps.set_ylabel(r'$\varepsilon$', color='cadetblue')

# Second row: Lagrange multiplier and plastic strain plots
ax_lam_eps_t = ax_lam.twinx()
ax_lam.plot(t_t, lam_t[:, 0, 0], 'o-', label=r'$\lambda$', color='purple')
ax_lam.set_xlabel(r'$t$')
ax_lam.set_ylabel(r'$\lambda$', color='purple')
ax_lam_eps_t.plot(t_t, eps_t, label=r'$\varepsilon$', color='green', ls='dashed')
ax_lam_eps_t.set_ylabel(r'$\varepsilon$', color='green')

ax_z_eps_p = ax_z.twinx()
ax_z.plot(t_t, z_t, label=r'$z$', color='orange')
ax_z.set_xlabel(r'$t$')
ax_z.set_ylabel(r'$Z$', color='orange')
ax_z_eps_p.plot(t_t, eps_p_t, label=r'$\varepsilon_\mathrm{p}$', color='blue', ls='dashed')
ax_z_eps_p.set_ylabel(r'$\varepsilon_\mathrm{p}$', color='blue')

plt.tight_layout()
plt.show()

## Monotonic stress-driven loading

In [ ]:
response = gsm.get_G_response(sig_t_strain, t_t, **mparams)
t_t, eps_ta, sig_ta, Eps_t, Sig_t, iter_t, lam_t, (d_t_t, d_eps_ta) = response
iter_t = iter_t[:, 0]
eps_t = eps_ta[:, 0, 0]
sig_t = sig_ta[:, 0]
eps_p_t, omega_t, z_t = Eps_t[:, 0, :].T
sig_p_t, Y_t, Z_t = Sig_t[:, 0, :].T
eps_t.shape, sig_t.shape

In [ ]:
fig, (ax_sig, ax_eps) = plt.subplots(1, 2, figsize=(12, 4))

# Plot stress-strain and iterations-strain curves
ax_iter = ax_sig.twinx()

ax_sig.plot(eps_t, sig_t, color='cadetblue', label=r'$\varepsilon$')
ax_sig.plot(eps_t-eps_p_t, sig_t, ls='dashed', color='cadetblue', label=r'$\varepsilon-\varepsilon^\mathrm{ve}$')
ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')
ax_sig.legend()

ax_iter.plot(eps_t, iter_t, ls='dashed', color='gray', lw=0.5)
ax_iter.set_ylabel(r'$k$', color='gray')

ax_eps.plot(t_t, eps_t, label=r'$\varepsilon$', color='cadetblue', lw=1)
ax_eps.plot(t_t, eps_p_t, label=r'$\varepsilon_\mathrm{vp}$', color='cadetblue', ls='dashed', lw=1)
ax_eps.legend()
ax_eps.set_ylabel(r'$\varepsilon$', color='cadetblue')

plt.tight_layout()
plt.show()

## Cyclic strain-driven loading

In [ ]:
t_, sig_max_, t_1_, t_2_, t_3_ = sp.symbols('t sig_max t_1 t_2 t_3', real=True, positive=True)
eps_t_ = sp.Piecewise(
    (sig_max_ / t_1_ * t_, t_ < t_1_),
    (sig_max_, t_ < t_2_),
    (sig_max_ - sig_max_/(t_3_-t_2_)*(t_ - t_2_), t_ < t_3_),
    (0, True)
)
get_eps_t = sp.lambdify((t_, sig_max_, t_1_, t_2_, t_3_), eps_t_, 'numpy')
final_time = 10
eps_max = 3
n_t = 151
#n_t = 31
t_t = np.linspace(0, final_time, n_t)
eps_n_t = get_eps_t(t_t, eps_max, 0.25*final_time, 0.5*final_time, 0.75*final_time)
fig, ax = plt.subplots(1,1)
ax.plot(t_t, eps_n_t)

In [ ]:
mparams = dict(E = 20, K = 5, S = 10, c = 1, r = 1, eps_0 = 0, f_c = 15, eta_vp=10)
response = gsm.get_response(eps_n_t, t_t, **mparams)
t_t, eps_ta, sig_ta, Eps_t, Sig_t, iter_t, lam_t, (d_t_t, d_eps_ta) = response
# Plot viscoplastic strain and damage along time
eps_t = eps_ta[:, 0]
eps_p_t, omega_t, z_t = Eps_t[:, 0, :].T
sig_p_t, Y_t, Z_t = Sig_t[:, 0, :].T

In [ ]:
fig, ((ax_sig, ax_omega), (ax_lam, ax_z)) = plt.subplots(2, 2, figsize=(12, 8))
fig.canvas.header_visible = False

# First row: Stress-strain and damage plots
ax_iter = ax_sig.twinx()
ax_eps = ax_omega.twinx()

ax_sig.plot(eps_t, sig_p_t, color='cadetblue', label=r'$\varepsilon$')
ax_sig.plot(eps_t-eps_p_t, Sig_t[:,:,0], ls='dashed', color='cadetblue', label=r'$\varepsilon-\varepsilon^\mathrm{ve}$')
ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')
ax_sig.legend()

ax_iter.plot(eps_t, iter_t, ls='dashed', color='gray', lw=0.5)
ax_iter.set_ylabel(r'$k$', color='gray')

ax_omega.plot(t_t, omega_t, label='Damage', color='brown')
ax_eps.plot(t_t, eps_t, label=r'$\varepsilon$', color='cadetblue', lw=1)
ax_eps.plot(t_t, eps_p_t, label=r'$\varepsilon_\mathrm{vp}$', color='cadetblue', ls='dashed', lw=1)
ax_eps.legend()
ax_omega.set_xlabel(r'$t$')
ax_omega.set_ylabel(r'$\omega$', color='brown')
ax_eps.set_ylabel(r'$\varepsilon$', color='cadetblue')

# Second row: Lagrange multiplier and plastic strain plots
ax_lam_eps_t = ax_lam.twinx()
ax_lam.plot(t_t, lam_t[:, 0, 0], 'o-', label=r'$\lambda$', color='purple')
ax_lam.set_xlabel(r'$t$')
ax_lam.set_ylabel(r'$\lambda$', color='purple')
ax_lam_eps_t.plot(t_t, eps_t, label=r'$\varepsilon$', color='green', ls='dashed')
ax_lam_eps_t.set_ylabel(r'$\varepsilon$', color='green')

ax_z_eps_p = ax_z.twinx()
ax_z.plot(t_t, Z_t, label=r'$z$', color='orange')
ax_z.set_xlabel(r'$t$')
ax_z.set_ylabel(r'$Z$', color='orange')
ax_z_eps_p.plot(t_t, eps_p_t, label=r'$\varepsilon_\mathrm{p}$', color='blue', ls='dashed')
ax_z_eps_p.set_ylabel(r'$\varepsilon_\mathrm{p}$', color='blue')

plt.tight_layout()
plt.show()